In [ ]:
import numpy as np
import pylab as plt
%matplotlib notebook

# plt.figure(figsize = (6,2))
# srate = 5000
# t = np.linspace(0,1,1*srate)
# plt.plot(t,sqsine(1,40,srate,post_ramp = 0.2))


In [ ]:
import threading
import nidaqmx
from nidaqmx.stream_writers import AnalogMultiChannelWriter
from nidaqmx.stream_readers import AnalogMultiChannelReader
from nidaqmx.stream_writers import DigitalMultiChannelWriter

def sqsine(duration,
           frequency,
           sampling_rate,
           pre_ramp = 0,
           post_ramp = 0,
           **kwargs):
    t = np.linspace(0,duration,int(duration*sampling_rate))
    waveform = np.sin(frequency*np.pi*t)**2
    ramp = np.ones_like(t)
    if pre_ramp>0:
        ramp[(t<(pre_ramp))] = (1/pre_ramp)*t[(t<(pre_ramp))] 
    if post_ramp>0:
        ramp[(t>(duration-post_ramp))] = (1/post_ramp)*t[(t<(post_ramp))][::-1] 
    return waveform*ramp

class TriggeredOptogeneticsWaveform():
    def __init__(self,
                 device = 'dev1',
                 waveform_channel = 0,
                 trigger_channel = None,
                 sampling_rate = None,
                 vrange = 10):
        self.device = device
        self.waveform_channel = waveform_channel
        self.trigger_channel = trigger_channel
        self.vrange = vrange
        self.sampling_rate = sampling_rate
        
        self.task_ao = nidaqmx.Task()
        self.task_ao.ao_channels.add_ao_voltage_chan('{0}/ao{1}'.format(self.device,self.waveform_channel),
                        min_val = -1*vrange,
                        max_val = vrange)
        if self.sampling_rate is None:
            self.sampling_rate = self.task_ao.timing.samp_clk_max_rate
        self.task_ao.timing.samp_clk_rate = self.sampling_rate
        self.waveform_parameters = None
        self.loaded = False
        
    def load(self,
             generator = 'sqsine',
             duration = 1,
             amplitude = 1,
             frequency = 40,
             pre_ramp = 0,
             post_ramp = 0.2,
             **kwargs):
        self.waveform = None
        self.loaded = False
        if generator == 'sqsine':
            self.waveform_parameters = dict(generator = generator,
                                            duration = duration,
                                            amplitude = amplitude,
                                            frequency = frequency,
                                            pre_ramp = pre_ramp,
                                            post_ramp = post_ramp,
                                            sampling_rate = self.sampling_rate,
                                            **kwargs)
            self.waveform = self.waveform_parameters['amplitude']*sqsine(**self.waveform_parameters)
        else:
            raise(ValueError('Could not load waveform. Unknown generator {0}'.format(generator)))
        #self.task_ao.export_signals.export_signal(nidaqmx.constants.Signal.SAMPLE_CLOCK, 'PFI0')
        self.task_ao.timing.cfg_samp_clk_timing(
            rate = self.sampling_rate,
            samps_per_chan = len(self.waveform))
        self.task_ao.write(self.waveform,auto_start=False)
        self.loaded = True
        
    def start(self):
        self.task_ao.start()
    def close(self):
        if not self.task_ao is None:
            self.task_ao.close()
            self.task_ao = None

In [ ]:
task = TriggeredOptogeneticsWaveform()
# task.load(frequency=40)

In [ ]:
task.load(frequency=40,amplitude = 4)

In [ ]:
task.task_ao.timing.cfg_samp_clk_timing(rate=5000,samps_per_chan=2)
task.task_ao.write([2.5,2.5],auto_start=True)

In [ ]:
task.start()

# # task.task_ao.wait_until_done()
# task.task_ao.stop()# nidaqmx.constants.SampleTimingType.SAMPLE_CLOCK
    

In [ ]:
task_ao.write(10*sqsine(1,100,5000,post_ramp = 0.2),auto_start=True)

In [ ]:
task.task_ao.write([2,2],auto_start=True)

In [ ]:
task_ao.write([1,1],auto_start=True)

In [ ]:
plt.plot(task.waveform)